## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-09-15-49-03 +0000,nyt,Live Updates: Parliament Burns in Nepal as Pro...,https://www.nytimes.com/live/2025/09/09/world/...
1,2025-09-09-15-47-30 +0000,nyt,Iran and U.N. Watchdog Meet to Discuss Resumin...,https://www.nytimes.com/2025/09/09/world/europ...
2,2025-09-09-15-46-14 +0000,bbc,"US created 911,000 fewer jobs than thought in ...",https://www.bbc.com/news/articles/cn8257jz8kvo...
3,2025-09-09-15-42-00 +0000,wsj,"U.S. added over 900,000 fewer jobs than previo...",https://www.wsj.com/economy/jobs/us-job-growth...
4,2025-09-09-15-41-41 +0000,startribune,J.J. McCarthy era makes dramatic fourth-quarte...,https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2357/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
72,trump,37
102,new,26
338,court,15
4,nepal,13
255,media,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
329,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...,125
285,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...,89
160,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...,83
133,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,81
186,2025-09-09-02-48-24 +0000,nyt,House Democrats Release Epstein’s Birthday Let...,https://www.nytimes.com/2025/09/08/us/politics...,80


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
329,125,2025-09-08-17-20-00 +0000,wsj,New polling by the Federal Reserve Bank of New...,https://www.wsj.com/economy/jobs/employment-su...
285,89,2025-09-08-20-12-00 +0000,nyt,Trump Issues Warning to Hamas as Israeli Milit...,https://www.nytimes.com/2025/09/08/world/middl...
160,83,2025-09-09-07-15-58 +0000,wapo,"At least 19 killed, hundreds injured in protes...",https://www.washingtonpost.com/world/2025/09/0...
161,53,2025-09-09-07-10-42 +0000,nypost,Bill Clinton letter in Epstein ‘birthday book’...,https://nypost.com/2025/09/09/us-news/bill-cli...
164,50,2025-09-09-06-41-09 +0000,bbc,Thai court rules ex-PM Thaksin must serve one ...,https://www.bbc.com/news/articles/cly7k2g37g4o...
232,47,2025-09-08-22-53-00 +0000,nypost,Lachlan Murdoch takes control of media empire ...,https://nypost.com/2025/09/08/media/lachlan-mu...
26,44,2025-09-09-15-13-22 +0000,wapo,Live updates: Israel’s attack in Qatar targets...,https://www.washingtonpost.com/world/2025/09/0...
324,39,2025-09-08-17-34-01 +0000,nypost,France’s government collapses in spectacular f...,https://nypost.com/2025/09/08/world-news/franc...
109,36,2025-09-09-10-36-47 +0000,nypost,‘Brutally savage’ Russian airstrike kills more...,https://nypost.com/2025/09/09/world-news/russi...
332,35,2025-09-08-17-02-01 +0000,nypost,Trump admin launches ‘Operation Midway Blitz’ ...,https://nypost.com/2025/09/08/us-news/trump-ad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
